In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import scipy
import pickle

In [4]:
# Load the required packages
import time
import psutil
import multiprocessing as mp

# Check the number of cores and memory usage
num_cores = mp.cpu_count()
print("This kernel has ",num_cores,"cores and you can find the information regarding the memory usage:",psutil.virtual_memory())

This kernel has  4 cores and you can find the information regarding the memory usage: svmem(total=8217075712, available=4517175296, percent=45.0, used=2986188800, free=2581528576, active=3877945344, inactive=1353682944, buffers=314216448, cached=2335141888, shared=428265472, slab=184950784)


In [5]:
data1 = pd.read_csv("./../netflix-prize-data/updated_data1.txt", sep = ",", header = None)
data1.columns = ["UserID", "Ratings", "Date", "MovieID"]
#print(data1)

In [6]:
print(data1.shape)

(24053764, 4)


In [7]:
data1 = data1.drop(data1[data1.UserID > 2000].index)

In [8]:
print(data1.shape)

(16543, 4)


In [9]:
Mean = data1.groupby(["UserID"], as_index = False, sort = False).mean().rename(columns = {"Ratings": "rating_mean"})[["UserID", "rating_mean"]]
data1 = pd.merge(data1, Mean, on = "UserID", how = "left", sort = False)
data1["Adjusted_Ratings"] = data1["Ratings"] - data1["rating_mean"]
print(data1)

       UserID  Ratings       Date  MovieID  rating_mean  Adjusted_Ratings
0         915        5  2005-08-1        1     4.285714          0.714286
1        1333        4  2004-05-1        3     2.676000          1.324000
2         967        1  2005-01-2        4     3.625000         -2.625000
3         685        3  2005-11-3        5     4.133333         -1.133333
4        1333        3  2005-04-0        8     2.676000          0.324000
5         695        1  2005-10-1        8     3.320988         -2.320988
6        1856        5  2005-06-2        8     4.425532          0.574468
7        1983        3  2005-04-1        8     3.740741         -0.740741
8           7        5  2005-07-3        8     4.066667          0.933333
9         307        3  2005-09-2        8     3.666667         -0.666667
10        734        1  2005-08-2        8     3.285714         -2.285714
11        742        4  2005-05-1        8     3.833333          0.166667
12        840        2  2005-08-0     

In [10]:
data1.isnull().any()

UserID              False
Ratings             False
Date                False
MovieID             False
rating_mean         False
Adjusted_Ratings    False
dtype: bool

In [11]:
distinct_users = np.unique(data1["UserID"])
corr = {}
for i in range(len(distinct_users)):
    #print(i)
    for j in range(i+1,len(distinct_users)):
        if j == len(data1):
            break
        #print(j)
        usr1 = distinct_users[i] #userid of i'th element of distinct_users array
        usr2 = distinct_users[j]

        data1_usr1 = data1[data1["UserID"]==usr1]   #usr1 all entries
        data1_usr2 = data1[data1["UserID"]==usr2]

        df3 = pd.merge(data1_usr1, data1_usr2, on=['MovieID'], how='inner')
        
        #print(df3.loc[df3['UserID_x'] == 6 ])
        
        nmr_sum = (df3["Adjusted_Ratings_x"]*df3["Adjusted_Ratings_y"]).sum()
        
        dmr_sum1 = np.sqrt((np.square(df3["Adjusted_Ratings_x"])).sum())
        dmr_sum2 = np.sqrt((np.square(df3["Adjusted_Ratings_y"])).sum())

        corr_pearson = nmr_sum / (dmr_sum1*dmr_sum2)
        
        corr[str(usr1)+"_"+str(usr2)] = corr_pearson
        
        print(str(usr1)+"_"+str(usr2),corr[str(usr1)+"_"+str(usr2)])
        
        

6_7 0.17356520434280956
6_8 0.30689745276171304
6_10 0.27395246578972887
6_25 -0.948683298050514
6_33 0.5291502622129179
6_42 0.5502892870679753
6_59 -0.05057836766016114
6_79 0.1482971409765376
6_83 -1.0
6_87 0.4374087583539865
6_94 0.0409574899723292
6_97 0.0581538628338123
6_116 0.42626362548451735
6_126 -1.0
6_130 -0.23543547789870944
6_131 -4.0390295281078565e-17
6_133 nan
6_134 0.04642600071970398
6_142 0.10827065342261297
6_149 -0.8682431421244595
6_158 0.3015113445777635
6_164 0.2332842396035758
6_168 0.5280786895875598
6_169 0.2608384798278157
6_177 1.0
6_178 0.8952737857857515
6_183 0.2149696841302725
6_188 0.31134459184302377
6_189 0.0
6_192 -0.17401950955164103
6_195 0.25378726933144974


/home/dipak/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: RuntimeWarning: invalid value encountered in double_scalars


6_199 0.13046431690413984
6_201 0.35712994918847774
6_215 nan
6_231 0.42172280741983037
6_242 0.06453861137337433
6_247 0.4230617087005147
6_248 -0.7071067811865476
6_250 0.4281095343040981
6_261 -0.25513125674454123
6_265 0.24392736353556368
6_266 0.22647233226139948
6_267 -1.0
6_268 0.5006635276266322
6_283 0.18274882872234424
6_291 0.41665251441196366
6_296 -0.13502333722647764
6_298 0.1830307574737877
6_299 0.8660254037844385
6_301 0.6667020965642104
6_302 0.19769242700693496
6_304 0.12744894313735392
6_305 -0.32100621643173516
6_307 -0.483045891539648
6_308 -0.03656362120635642
6_310 0.15257189753323228
6_312 0.6067798762169178
6_314 -0.4753072873819255
6_330 -0.6693975760754364
6_331 -0.18257418583505547
6_333 0.5568846567109816
6_352 0.12524921782015075
6_358 0.20274687307477895
6_363 -1.0
6_368 0.8139943934615719
6_369 0.48338026580308086
6_379 -0.4961389383568339
6_383 0.2101188207088983
6_384 0.3307930438680391
6_385 -0.9333456062030595
6_392 0.1957777118745734
6_413 0.285830

7_242 0.28058165351622405
7_247 0.19822272086135467
7_248 0.0
7_250 0.1318454423964495
7_261 0.30319565796607334
7_265 0.24796812727567474
7_266 0.296307780379997
7_267 -0.509802390301733
7_268 0.18580185161353585
7_283 0.1861260402067955
7_291 0.4772661321051989
7_296 0.1514253375873004
7_298 0.24977214776830123
7_299 0.0
7_301 0.08015248845467378
7_302 0.2448515090318912
7_304 0.05617989169116024
7_305 -0.05303314868570294
7_307 -0.0821341627100438
7_308 0.9240168324218909
7_310 0.21280785424799095
7_312 0.0
7_314 0.000675474642102229
7_330 -0.0724229845980066
7_331 -0.05044332723053164
7_333 0.44598154875430546
7_352 -0.19307454120681156
7_358 0.34436321265441056
7_363 0.9662217301890476
7_368 -0.26842722077717096
7_369 0.04431043404281033
7_379 -0.9221943818285308
7_383 0.05261700102817534
7_384 0.27577840188131386
7_385 1.0
7_392 0.3018712187593393
7_413 0.3219738813589541
7_416 nan
7_424 -0.030471067406847453
7_437 0.3179336630616901
7_439 -0.6322563238539187
7_440 0.480696379672

8_250 0.6976514428272618
8_261 nan
8_265 0.6242463443461822
8_266 0.5323045903426588
8_267 nan
8_268 -0.14951259010043316
8_283 0.06419024335625924
8_291 0.5614346140443459
8_296 -0.17833148948332928
8_298 0.7159480058386265
8_299 0.7071067811865475
8_301 0.8161881780532954
8_302 0.22941824838760133
8_304 0.6233526148864096
8_305 -0.8699701179439605
8_307 nan
8_308 nan
8_310 nan
8_312 0.7071067811865475
8_314 -0.42251818641587696
8_330 -0.3665245183824905
8_331 nan
8_333 0.630528288755463
8_352 -0.11359236684941296
8_358 -0.11955823842746055
8_363 nan
8_368 nan
8_369 1.0
8_379 -0.9486832980505138
8_383 -0.09346229556907262
8_384 -0.08703882797784893
8_385 -1.0
8_392 nan
8_413 0.7071067811865476
8_416 nan
8_424 0.33333333333333354
8_437 0.5477000749349096
8_439 1.0
8_440 0.7071067811865476
8_442 nan
8_453 nan
8_462 -0.08613410570607252
8_466 0.0
8_470 -0.17924428597928826
8_471 nan
8_477 0.24878173294843997
8_478 -0.021546520679428506
8_479 nan
8_481 0.1280075045319251
8_485 0.0
8_490 n

...

1717_1771 -0.23380869715030048
1717_1779 nan
1717_1789 1.0
1717_1791 1.0
1717_1798 -0.0747519199193538
1717_1801 1.0
1717_1809 0.058962365165913
1717_1810 -0.6033086550339456
1717_1811 0.08941180472342485
1717_1818 1.0
1717_1819 0.0
1717_1830 0.6840491424450842
1717_1831 0.511156112683622
1717_1846 -0.3898115462384199
1717_1856 0.4473547338157438
1717_1871 0.43450592374043845
1717_1874 nan
1717_1878 1.0
1717_1882 0.22903933372554655
1717_1888 nan
1717_1889 1.0
1717_1890 nan
1717_1893 0.34184648371461795
1717_1894 0.35647394842244057
1717_1896 nan
1717_1897 -0.10243264638563807
1717_1900 0.2606357623651077
1717_1911 -0.5679872117781536
1717_1912 nan
1717_1913 -1.0
1717_1918 0.10030502931159604
1717_1921 0.38289896681459
1717_1922 -0.606408613396878
1717_1928 1.0
1717_1933 0.822086753889433
1717_1934 nan
1717_1940 -1.0
1717_1942 nan
1717_1944 -1.0
1717_1947 0.20469049520290458
1717_1969 0.7668994490222011
1717_1982 1.0
1717_1983 -0.12647544878244382
1717_1985 0.65506138594274
1717_1994 -

1736_1897 -0.4539019021907726
1736_1900 -0.4817930983478328
1736_1911 1.0
1736_1912 nan
1736_1913 nan
1736_1918 -0.3903453184137668
1736_1921 -0.8715755371245495
1736_1922 -0.3110765624019272
1736_1928 -1.0
1736_1933 -0.7476309016255966
1736_1934 nan
1736_1940 nan
1736_1942 -1.0
1736_1944 nan
1736_1947 -0.44848969499919716
1736_1969 0.4678877204190325
1736_1982 1.0
1736_1983 -0.7174634171770226
1736_1985 -0.9098103736717087
1736_1994 -1.0
1736_2000 -0.47448367673868713
1738_1746 0.2425681132152973
1738_1750 -0.42619694463526886
1738_1753 nan
1738_1760 0.3540826149662496
1738_1768 0.45107398568019086
1738_1771 0.21783665497250299
1738_1779 nan
1738_1789 -0.5683605986035063
1738_1791 0.8350293597017083
1738_1798 0.03356681264466124
1738_1801 0.9982743731749958
1738_1809 0.10470358282093153
1738_1810 0.21859681924195173
1738_1811 0.15836079841686787
1738_1818 0.45680468499262766
1738_1819 -0.9977851578566087
1738_1830 0.031315194024792166
1738_1831 0.30045783513398405
1738_1846 0.24316171

1779_1846 nan
1779_1856 nan
1779_1871 nan
1779_1874 nan
1779_1878 nan
1779_1882 nan
1779_1888 nan
1779_1889 nan
1779_1890 nan
1779_1893 nan
1779_1894 nan
1779_1896 nan
1779_1897 nan
1779_1900 nan
1779_1911 nan
1779_1912 nan
1779_1913 nan
1779_1918 nan
1779_1921 nan
1779_1922 nan
1779_1928 nan
1779_1933 nan
1779_1934 nan
1779_1940 nan
1779_1942 nan
1779_1944 nan
1779_1947 nan
1779_1969 nan
1779_1982 nan
1779_1983 nan
1779_1985 nan
1779_1994 nan
1779_2000 nan
1789_1791 0.7996723699502313
1789_1798 -0.8119623276481978
1789_1801 nan
1789_1809 0.009392767477367838
1789_1810 1.0
1789_1811 -0.7123160636154746
1789_1818 1.0
1789_1819 nan
1789_1830 -0.2474029833934953
1789_1831 0.13814703962882846
1789_1846 -0.37306672906249494
1789_1856 -0.6013700843979107
1789_1871 -0.320480545011662
1789_1874 nan
1789_1878 0.9377487607237037
1789_1882 1.0
1789_1888 nan
1789_1889 -1.0
1789_1890 nan
1789_1893 nan
1789_1894 -0.637226812519971
1789_1896 nan
1789_1897 -0.2285583755252324
1789_1900 0.3025391935338

1830_1940 -0.4434910676358664
1830_1942 -0.6895351883783316
1830_1944 -0.26945610091006905
1830_1947 0.5037063242228635
1830_1969 -0.9086768439633128
1830_1982 0.11566410850541169
1830_1983 -0.19573712948887112
1830_1985 0.43513665045976047
1830_1994 0.9754552628518156
1830_2000 -0.04538739024818558
1831_1846 0.1634209445378268
1831_1856 0.3894835556282409
1831_1871 0.32164980861390435
1831_1874 nan
1831_1878 -0.293455442381909
1831_1882 -0.5228050144259127
1831_1888 0.057539564560923184
1831_1889 -0.471118797538997
1831_1890 0.24380234087004982
1831_1893 0.7387032527030919
1831_1894 0.04553667838455555
1831_1896 -1.0
1831_1897 0.24215192250564083
1831_1900 0.3687859551980387
1831_1911 0.183502429392911
1831_1912 -1.0
1831_1913 0.506803773144885
1831_1918 -0.043490637870125096
1831_1921 -0.46325992781083103
1831_1922 -0.16662727683677447
1831_1928 1.0
1831_1933 0.023356959059601518
1831_1934 0.9989915540104686
1831_1940 -0.841407312617695
1831_1942 0.9335284508396899
1831_1944 -0.67136

1911_1933 0.5547001962252291
1911_1934 -1.0
1911_1940 1.0
1911_1942 nan
1911_1944 1.0
1911_1947 -0.13546935991255898
1911_1969 nan
1911_1982 0.9761870601839526
1911_1983 0.8954962634705147
1911_1985 0.17248597171142824
1911_1994 nan
1911_2000 0.5247282618559876
1912_1913 1.0
1912_1918 nan
1912_1921 nan
1912_1922 nan
1912_1928 nan
1912_1933 nan
1912_1934 nan
1912_1940 nan
1912_1942 1.0
1912_1944 nan
1912_1947 1.0
1912_1969 1.0
1912_1982 1.0
1912_1983 nan
1912_1985 nan
1912_1994 nan
1912_2000 nan
1913_1918 0.057154491870687225
1913_1921 1.0
1913_1922 0.03896518385497673
1913_1928 nan
1913_1933 nan
1913_1934 nan
1913_1940 nan
1913_1942 1.0
1913_1944 nan
1913_1947 0.7688751859179731
1913_1969 0.03903923371429276
1913_1982 -0.4862807188412806
1913_1983 -1.0
1913_1985 0.48004501079806694
1913_1994 -1.0
1913_2000 0.040624077129486284
1918_1921 0.1979426493973183
1918_1922 0.08231221243477867
1918_1928 0.5038009447262912
1918_1933 0.5476042788512288
1918_1934 nan
1918_1940 -0.3076534836325802


In [12]:
len(corr)

61776

In [13]:
def isNaN(num):
    return num != num

In [14]:
for key in corr:
    if isNaN(corr[key]):
        corr[key] = 0

In [15]:
#Drop to a pickle file

def storeData(data, x): 
    # Its important to use binary mode 
    dbfile = open(x, 'ab+') 
      
    # source, destination 
    pickle.dump(corr, dbfile)                      
    dbfile.close() 

In [16]:
storeData(corr, 'Pearson_Correlations')

In [17]:
data1.to_pickle("User_data.pkl")